In [ ]:
import numpy as np
import pandas as pd
from scapy.all import IP, rdpcap

In [ ]:
def resize(arr, size):
    if len(arr) >= size:
        return arr[0:size]
    return np.pad(arr,(0,size - len(arr)))

In [ ]:
data_frame = list()  # List to store the processed data
local_ip = '192.168.1.111'  # Local IP address
resize_len = 10000  # Length to resize the data
input_path = 'E:/4tab-traffic/'  # Path to input .pcap files
count_repeat = 10  # Number of repetitions

# Read combinations of A, B, C, and D from the CSV file
combinations = pd.read_csv('E:/4taburl-label.csv')

# Loop through each combination of A, B, C, and D
for index, row in combinations.iterrows():
    A, B, C, D = row['A'], row['B'], row['C'], row['D']
    
    # Loop for each repetition
    for F in range(count_repeat):
        file_name = f'{input_path}{int(A)}-{int(B)}-{int(C)}-{int(D)}-{F}.pcap'  # Construct file name
        
        try:
            packets = rdpcap(file_name)  # Read pcap file
            data = [A, B, C, D, F]  # Initialize data list with A, B, C, D, and repetition index
            
            # Process each packet in the pcap file
            for packet in packets:
                if IP in packet:
                    ip_src = packet[IP].src
                    ip_dst = packet[IP].dst

                    # Append packet length based on source or destination IP
                    if ip_src == local_ip:
                        data.append(len(packet))
                    elif ip_dst == local_ip:
                        data.append(-len(packet))

            data_list = np.array(data)  # Convert data to numpy array
            data_list = resize(data_list, resize_len + 5)  # Resize data array
            data_list = np.array([data_list])  # Convert data list to array

            # Append to data_frame if not empty, otherwise initialize it
            if len(data_frame) == 0:
                data_frame = data_list
            else:
                data_frame = np.append(data_frame, data_list, axis=0)
        except Exception as e:
            print(f'\tError opening file {file_name}:', e)  # Print error if file can't be opened
            pass

    print(f'\rprocessing: {A}-{B}-{C}-{D}', end='')  # Print current combination being processed

print('\ndone!')

out_path = 'E:/chrome/'  # Output path
df_data = pd.DataFrame(data_frame)  # Convert data to pandas DataFrame
df_data.to_csv(out_path + 'chrome-4tab.csv', index=None, header=None)  # Save DataFrame to CSV
